# 17. Representation Learning and Generative Learning Using Autoencoders and GANs

Autoencoders are artificial neural networks capable of learning dense representations of the input data, called **latent representations** or **codings**, without any supervision.  

These codings typically have a much lower dimensionality than the input data, making autoencoders useful for: 
* Dimensionality reduction (esp. visualization)
* Feature detection (pretraining of DNN)
* Some of them are generative models 

Generative Adversarial Network (GANs) are a new class of generative models already widely adopted, with applications in image editing and creation, super resolution, coloring etc.

Key differences between the two, in short:

1. Autoencoder are made of an encoder and a decoder. Codings are byproducts of the autoencoder learning the identity function under some constraints. 
2. GANs, on the other hand, are made of a generator and discriminator.

### Efficient Data Representations

An autoencoder looks at the inputs, converts them to an efficient latent representation, and then spits out something that (hopefully) looks very close to the inputs.

Autoencoder outputs are called **reconstructions**, and the cost function contains a **reconstruction loss** that penalizes the model when the reconstructions are different from the inputs. Because of lower dimensionality, the model is said to be **undercomplete**. 

### Performing PCA with an Undercomplete Linear Autoencoder

If the autoencoder uses only linear activations and the cost function is the mean squared error (MSE), then it ends up performing Principal Component Analysis (PCA).

In [1]:
from tensorflow import keras

encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape=[2])])
autoencoder = keras.models.Sequential([encoder, decoder])
autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.1))

This looks actually very similar to a MLP. Now let’s train the model on a simple generated 3D dataset and use it to encode that same dataset (in 2D): 

In [2]:
history = autoencoder.fit(X_train, X_train, epochs=20)
codings = encoder.predict(X_train)

NameError: name 'X_train' is not defined

### Stacked Autoencoders

